<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/Tensorflow2.x/Tensorflow2.x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
import timeit
def linear_layer(x):
  return 3 * x + 2

@tf.function
def simple_nn(x):
  return tf.nn.relu(linear_layer(x))

def simple_function(x):
  return 3 * x

#simple_nn #<tensorflow.python.eager.def_function.Function at 0x7f9736390ad0>
#simple_function #<function __main__.simple_function>

cell = tf.keras.layers.LSTMCell(100)

@tf.function
def fn(input, state):
  return cell(input,state)

input = tf.zeros([100,100])
state = [tf.zeros([100,100])] * 2

#워밍업
cell(input,state)
fn(input,state)

graph_time = timeit.timeit(lambda: cell(input,state), number=100)
auto_graph_time = timeit.timeit(lambda: fn(input,state), number=100)
print('graph time :', graph_time)
print('auto graph time :', auto_graph_time)

graph time : 0.12537865199999487
auto graph time : 0.10789341600002444
